In [1]:
from data_load import mixture
import tensorflow as tf
import numpy as np
path = '/Users/smaity/projects/mnist-label-shift/MNIST'

In [2]:
x_source, y_source = mixture(12000, 0.9, flatten=False)
x_target, y_target = mixture(1000, 0.2, flatten=False)
x_source.shape, y_source.shape

((12000, 28, 28), (12000,))

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20)

Epoch 1/20
375/375 [==============================] - 1s 4ms/step - loss: 4.0128 - accuracy: 0.8884
Epoch 2/20
375/375 [==============================] - 1s 3ms/step - loss: 0.7888 - accuracy: 0.9227
Epoch 3/20
375/375 [==============================] - 1s 3ms/step - loss: 0.4457 - accuracy: 0.9358
Epoch 4/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2834 - accuracy: 0.9478
Epoch 5/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2051 - accuracy: 0.9531
Epoch 6/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1837 - accuracy: 0.9572
Epoch 7/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1259 - accuracy: 0.9656
Epoch 8/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1067 - accuracy: 0.9706
Epoch 9/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0939 - accuracy: 0.9708
Epoch 10/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0866 - accuracy: 0.9758

In [4]:
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 1.0599 - accuracy: 0.8560


[1.0599126815795898, 0.8560000061988831]

In [16]:
w = model.get_weights()
bias = np.log(0.2/0.8) - np.log(0.9/0.1)
print(w[-1])
w[-1][0], w[-1][1] = w[-1][0] - bias, w[-1][1] + bias
print(w[-1])

[ 0.13651039 -0.13651043]
[ 3.7200294 -3.7200294]


In [17]:
model.set_weights(w)
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.5624 - accuracy: 0.9240


[0.5623974204063416, 0.9240000247955322]

In [7]:
y_predict = model.predict_classes(x_target)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [8]:
from sklearn import metrics
m = 12000
reg = 0
prop_source = 0.9
confusion_matrix = metrics.confusion_matrix(model.predict_classes(x_source),y_source, labels=[0,1])/m
prop_target = np.mean(model.predict_classes(x_target))
    
xi = np.array([1-prop_target,prop_target])
w = np.matmul(np.linalg.inv(confusion_matrix + reg*np.diag([1, 1])),xi)
prop_targets = w*np.array([1-prop_source, prop_source])
if prop_targets[0] <0:
    prop_targets[0] = 0
if prop_targets[1] <0:
    prop_targets[1] = 0
prop_targets = prop_targets/np.sum(prop_targets)

In [9]:
prop_targets

array([0.71632497, 0.28367503])

In [10]:
sample_weight_0 = prop_targets[0]/0.1
sample_weight_1 = prop_targets[1]/0.9
sample_weight = sample_weight_0 * (1-y_source) + sample_weight_1 * y_source


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20, sample_weight = sample_weight)


Epoch 1/20
375/375 [==============================] - 0s 1ms/step - loss: 8.1652 - accuracy: 0.5821
Epoch 2/20
375/375 [==============================] - 0s 1ms/step - loss: 2.0375 - accuracy: 0.7150
Epoch 3/20
375/375 [==============================] - 0s 1ms/step - loss: 1.0845 - accuracy: 0.7653
Epoch 4/20
375/375 [==============================] - 0s 1ms/step - loss: 0.7853 - accuracy: 0.7842
Epoch 5/20
375/375 [==============================] - 1s 2ms/step - loss: 0.5291 - accuracy: 0.8217
Epoch 6/20
375/375 [==============================] - 1s 2ms/step - loss: 0.4442 - accuracy: 0.8380
Epoch 7/20
375/375 [==============================] - 1s 2ms/step - loss: 0.3723 - accuracy: 0.8660
Epoch 8/20
375/375 [==============================] - 1s 2ms/step - loss: 0.3450 - accuracy: 0.8673
Epoch 9/20
375/375 [==============================] - 1s 2ms/step - loss: 0.2791 - accuracy: 0.8848
Epoch 10/20
375/375 [==============================] - 1s 1ms/step - loss: 0.2620 - accuracy: 0.8833

In [11]:
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.7665 - accuracy: 0.8930


[0.7665160298347473, 0.8930000066757202]

In [12]:
sample_weight_0 = 1/0.1
sample_weight_1 = 1/0.9
sample_weight = sample_weight_0 * (1-y_source) + sample_weight_1 * y_source


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20, sample_weight = sample_weight)

Epoch 1/20
375/375 [==============================] - 1s 1ms/step - loss: 20.4894 - accuracy: 0.7413
Epoch 2/20
375/375 [==============================] - 0s 1ms/step - loss: 4.3206 - accuracy: 0.8171
Epoch 3/20
375/375 [==============================] - 1s 1ms/step - loss: 2.0715 - accuracy: 0.8593
Epoch 4/20
375/375 [==============================] - 1s 1ms/step - loss: 1.6824 - accuracy: 0.8769
Epoch 5/20
375/375 [==============================] - 1s 1ms/step - loss: 1.1660 - accuracy: 0.8928
Epoch 6/20
375/375 [==============================] - 0s 1ms/step - loss: 0.7923 - accuracy: 0.9162
Epoch 7/20
375/375 [==============================] - 0s 1ms/step - loss: 0.7592 - accuracy: 0.9192
Epoch 8/20
375/375 [==============================] - 0s 1ms/step - loss: 0.4956 - accuracy: 0.9363
Epoch 9/20
375/375 [==============================] - 0s 1ms/step - loss: 0.5518 - accuracy: 0.9368
Epoch 10/20
375/375 [==============================] - 0s 1ms/step - loss: 0.6094 - accuracy: 0.931

In [13]:
m = model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 1.0555 - accuracy: 0.8600


In [14]:
m[1]

0.8600000143051147

In [15]:
from mnist_label_shift import label_shift
label_shift()

ValueError: could not convert string to float: '--ip=127.0.0.1'

In [3]:
import itertools
n_targets = [1000, 5000, 10000]
pi_targets = [0.05, 0.1, 0.2, 0.4]
iters = range(20)
l = list(itertools.product(n_targets, pi_targets, iters))

In [4]:
l

[(1000, 0.05, 0),
 (1000, 0.05, 1),
 (1000, 0.05, 2),
 (1000, 0.05, 3),
 (1000, 0.05, 4),
 (1000, 0.05, 5),
 (1000, 0.05, 6),
 (1000, 0.05, 7),
 (1000, 0.05, 8),
 (1000, 0.05, 9),
 (1000, 0.05, 10),
 (1000, 0.05, 11),
 (1000, 0.05, 12),
 (1000, 0.05, 13),
 (1000, 0.05, 14),
 (1000, 0.05, 15),
 (1000, 0.05, 16),
 (1000, 0.05, 17),
 (1000, 0.05, 18),
 (1000, 0.05, 19),
 (1000, 0.1, 0),
 (1000, 0.1, 1),
 (1000, 0.1, 2),
 (1000, 0.1, 3),
 (1000, 0.1, 4),
 (1000, 0.1, 5),
 (1000, 0.1, 6),
 (1000, 0.1, 7),
 (1000, 0.1, 8),
 (1000, 0.1, 9),
 (1000, 0.1, 10),
 (1000, 0.1, 11),
 (1000, 0.1, 12),
 (1000, 0.1, 13),
 (1000, 0.1, 14),
 (1000, 0.1, 15),
 (1000, 0.1, 16),
 (1000, 0.1, 17),
 (1000, 0.1, 18),
 (1000, 0.1, 19),
 (1000, 0.2, 0),
 (1000, 0.2, 1),
 (1000, 0.2, 2),
 (1000, 0.2, 3),
 (1000, 0.2, 4),
 (1000, 0.2, 5),
 (1000, 0.2, 6),
 (1000, 0.2, 7),
 (1000, 0.2, 8),
 (1000, 0.2, 9),
 (1000, 0.2, 10),
 (1000, 0.2, 11),
 (1000, 0.2, 12),
 (1000, 0.2, 13),
 (1000, 0.2, 14),
 (1000, 0.2, 15),
 (